##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introducción a las redes RL y Deep Q

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/0_intro_rl"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/0_intro_rl.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/0_intro_rl.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/0_intro_rl.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a> </td>
</table>

## Introducción

El Aprendizaje por Refuerzo (RL) es un marco general en el que los agentes aprenden a ejecutar acciones en un entorno con el fin de maximizar una recompensa. Los dos componentes principales son el entorno, que representa el problema a resolver, y el agente, que representa el algoritmo de aprendizaje.

El agente y el entorno interactúan constantemente entre sí. En cada paso de tiempo, el agente toma una acción en el entorno con base en su *política* $\pi(a_t|s_t)$, donde $s_t$ es la observación actual del entorno, y recibe una recompensa $r_{t+1}$ y la siguiente observación $s_{t+1}$ del entorno. El objetivo es mejorar la política para maximizar la suma de recompensas (rendimiento).

Nota: Es importante distinguir entre el `state` del entorno y la `observation`, que es la parte del `state` del entorno que el agente puede ver, por ejemplo, en un juego de póquer, el estado del entorno está compuesto por las cartas de todos los jugadores y las cartas de la comunidad, pero el agente solo puede observar sus propias cartas y algunas de las cartas de la comunidad. En la mayoría de la literatura, estos términos se utilizan indistintamente y la observación también se denota como $s$.

![Bucle de interacción agente-entorno](images/rl_overview.png)

Se trata de un marco muy general y puede modelar una gran variedad de problemas de toma de decisiones secuenciales, como juegos, robótica, etc.


## El entorno Cartpole

El entorno Cartpole es uno de los problemas clásicos de aprendizaje por refuerzo más conocidos (el *"¡Hola, mundo!"* del RL). Hay un poste unido a un carro, que puede moverse a lo largo de una pista sin fricción. El poste comienza en posición vertical y el objetivo es evitar que se caiga controlando el carro.

- La observación del entorno $s_t$ es un vector en 4D que representa la posición y la velocidad del carro, y el ángulo y la velocidad angular del poste.
- El agente puede controlar el sistema al tomar una de 2 medidas $a_t$: empujar el carro a la derecha (+1) o a la izquierda (-1).
- Se proporciona una recompensa $r_{t+1} = 1$ por cada paso de tiempo que el poste permanezca en posición vertical. El episodio termina cuando se cumple una de las siguientes condiciones:
    - el poste se inclina por encima de algún ángulo límite
    - el carro se mueve fuera de los límites del mundo
    - pasan 200 pasos de tiempo.

El objetivo del agente es aprender una política $\pi(a_t|s_t)$ para maximizar la suma de recompensas de un episodio \sum_{t=0}^{T} \gamma^t r_t$. Aquí $\gamma$ es un factor de descuento en $[0, 1]$ que descuenta las recompensas futuras en relación con las inmediatas. Este parámetro nos ayuda a centrar la política, de manera que se preocupe más por obtener recompensas rápidamente.


## El agente DQN

El [algoritmo DQN (Deep Q-Network)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) fue desarrollado en 2015 por DeepMind. Fue capaz de resolver una amplia gama de juegos Atari (algunos a un nivel de superhombre) gracias a la combinación de aprendizaje por refuerzo y redes neuronales profundas a escala. El algoritmo se desarrolló mejorando un algoritmo clásico de RL llamado Q-Learning con redes neuronales profundas y una técnica conocida como *repetición de experiencia*.

### Q-Learning

Q-Learning se basa en la noción de función Q. La función Q (también conocida como función de valor estado-acción) de una política $\pi$, $Q^{\pi}(s, a)$, mide el rendimiento esperado o la suma descontada de las recompensas obtenidas a partir del estado $s$ al tomar primero la acción $a$ y seguir después por la política $\pi$. Definimos la función Q óptima $Q^*(s, a)$ como el rendimiento máximo que puede obtenerse a partir de la observación $s$, ejecutando la acción $a$ y siguiendo la política óptima a partir de entonces. La función Q óptima obedece a la siguiente ecuación de optimalidad de *Bellman*:

```
$\begin{equation}Q^\ast(s, a) = \mathbb{E}[ r + \gamma \max_{a'} Q^\ast(s', a') ]\end{equation}$
```

Esto significa que el rendimiento a partir del estado $s$ y la acción $a$ corresponde a la suma de recompensas inmediatas $r$ y el rendimiento (descontado por $\gamma$) que se obtiene al seguir la política óptima desde allí en adelante hasta el final del episodio (es decir, la recompensa máxima del siguiente estado $s'$). La expectativa se calcula tanto sobre la distribución de recompensas inmediatas $r$ como sobre los posibles estados posteriores $s'$.

La idea básica de Q-Learning consiste en utilizar la ecuación de optimalidad de Bellman como actualización iterativa $Q_{i+1}(s, a) \leftarrow \mathbb{E}\left[ r + \gamma \max_{a'} Q_{i}(s', a')\right]$, y se puede demostrar que esto converge a la función óptima $Q$, es decir, $Q_i \rightarrow Q^*$ como $i \rightarrow \infty$ (consulte el [estudio sobre DQN](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)).

### Deep Q-Learning

Para la mayoría de los problemas, no resulta práctico representar la función $Q$ como una tabla con valores para cada combinación de $s$ y $a$. Por ello, entrenamos un aproximador de funciones, como una red neuronal con parámetros $\theta$, para estimar los valores de Q$, es decir, $Q(s, a; \theta) \approx Q^*(s, a)$. Esto puede hacerse mediante la minimización de la siguiente pérdida en cada paso $i$:

$\begin{equation}L_i(\theta_i) = \mathbb{E}{em0}{s, a, r, s'\sim \rho(.)} \left[ (y_i - Q(s, a; \theta_i))^2 \right]\end{equation}$ donde $y_i = r + \gamma \max{/em0}{a'} Q(s', a'; \theta_{i-1})$

Aquí, $y_i$ se denomina objetivo de TD (diferencia temporal), y $y_i - Q$ se denomina error de TD. $\rho$ representa la distribución del comportamiento, la distribución sobre las transiciones ${s, a, r, s'}$ obtenidas del entorno.

Tenga en cuenta que los parámetros de la iteración anterior $\theta_{i-1}$ son fijos y no se actualizan. En la práctica se utiliza una instantánea de los parámetros de la red de unas pocas iteraciones atrás en lugar de la última iteración. Esta copia se denomina *red objetivo*.

Q-Learning es un algoritmo *off-policy* que aprende sobre la política codiciosa $a = \max_{a} Q(s, a; \theta)$ mientras emplea una política de comportamiento diferente para actuar en el entorno o recopilar datos. Esta política de comportamiento generalmente es una política $\epsilon$-greedy que selecciona una acción codiciosa con probabilidad $1-\epsilon$ y una acción aleatoria con probabilidad $\epsilon$ para garantizar una buena cobertura del espacio estado-acción.

### Repetición de experiencia

Para evitar la necesidad de calcular la expectativa completa en la pérdida DQN, podemos minimizarla mediante el uso del descenso de gradiente estocástico. Si la pérdida se calcula solo con la última transición ${s, a, r, s'}$, esto se reduce al Q-Learning estándar.

El estudio DQN de Atari introdujo una técnica llamada Repetición de Experiencia para que las actualizaciones de red fueran más estables. En cada paso de tiempo de la recopilación de datos, las transiciones se agregan a un búfer circular denominado *búfer de repetición*. Luego, durante el entrenamiento, en lugar de usar solo la última transición para calcular la pérdida y su gradiente, lo calculamos con ayuda de un minilote de transiciones extraídas del búfer de repetición. Esto tiene dos ventajas: una mayor eficiencia de los datos que se genera al reutilizar cada transición en muchas actualizaciones, y una mayor estabilidad como consecuencia de utilizar transiciones no correlacionadas en un lote.


## DQN en Cartpole con TF-Agents

TF-Agents ofrece todos los componentes necesarios para entrenar un agente DQN, como el mismísimo agente, el entorno, las políticas, las redes, los búferes de repetición, los bucles de recopilación de datos y las métricas. Estos componentes se implementan como funciones de Python u operaciones gráficas de TensorFlow, y también disponemos de envoltorios para realizar conversiones entre ellos. Además, TF-Agents es compatible con el modo TensorFlow 2.0, que nos permite utilizar TF en modo imperativo.

A continuación, echemos un vistazo al [tutorial de entrenamiento del agente DQN en el entorno Cartpole mediante el uso de TF-Agents](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb).
